# Forecasting for SEO Traffic using the GA4 Api

In [ ]:
pip install pandas numpy matplotlib google-analytics-data oauth2client prophet

In [ ]:
import os
import numpy as np
import pandas as pd
from prophet import Prophet
from google.colab import drive
from prophet.plot import plot_plotly, plot_components_plotly
from oauth2client.service_account import ServiceAccountCredentials
from google.analytics.data_v1beta import BetaAnalyticsDataClient
from google.analytics.data_v1beta.types import DateRange, Dimension, Metric, RunReportRequest

In [ ]:
drive.mount('/content/drive')
client_secrets = '/content/drive/MyDrive/GCP/service_account.json'

In [13]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = client_secrets
property_id = 0
web_name=""

client = BetaAnalyticsDataClient()

In [15]:
def make_a_request(start_date, end_date, dimension, metric):
    request = RunReportRequest(
        property=f"properties/{property_id}",
        dimensions=[Dimension(name=dimension)],
        metrics=[Metric(name=metric)],
        date_ranges=[DateRange(start_date=start_date, end_date=end_date)],
    )
    response = client.run_report(request)
    return response

In [ ]:
response = make_a_request("2021-01-01", "2021-11-30", "date", "activeUsers")
output = []
for row in response.rows:
    output.append(
        {
            'ds': row.dimension_values[0].value,
            'y': float(row.metric_values[0].value)
        }
    )

users_df = pd.DataFrame.from_dict(output)

In [ ]:
m = Prophet()
m.fit(users_df)
future = m.make_future_dataframe(periods=12)
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
plot_plotly(m, forecast)